In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

from fleck import Star, generate_spots

In [ ]:
def emerge_decay(phases, lc, phase_max, emergence_timescale, decay_timescale):
    """
    Create *very* simple linear emergence/decay pattern for spot modulation
    
    Parameters
    ----------
    emergence_timescale : float
        timescale of spot emergence
    decay_timescale : float
        timescale of spot decay
    
    Returns
    -------
    lc2 : `~numpy.ndarray`
        Light curve convolved with the emergence/decay 
        envelope
    """
    
    emerge_slope = 1 / emergence_timescale / phases.unit
    decay_slope = -1 / decay_timescale / phases.unit
    
    y1 = 1 + emerge_slope * (phases - phase_max * phases.unit)
    y2 = 1 + decay_slope * (phases - phase_max * phases.unit)
    
    phase_max_ind = np.argmin(np.abs(phases - phase_max * phases.unit))
    env = np.concatenate([y1[:phase_max_ind], y2[phase_max_ind:]])
    env = np.max([np.zeros_like(env), env], axis=0)
    
    return 1 - env * (1 - lc)

def combine_lcs(lcs2):
    """
    Combine light curves in array ``lcs2`` along the 1th axis.
    
    Parameters
    ----------
    lcs2 : `~numpy.ndarray`
        Light curves
    """
    lcs2_less_than_unity = lcs2 < 1
    all_less = np.all(lcs2_less_than_unity, axis=1)
    any_less = np.any(lcs2_less_than_unity, axis=1)

    lcs3 = np.ones(lcs2.shape[0])
    lcs3[any_less] = 1 - np.add.reduce(1 - lcs2[any_less], axis=1)
    
    return lcs3

In [ ]:
# Make plots reproducible
np.random.seed(42)

# Set spot contrast, limb-darkening parameters
spot_contrast = 0.7
u_ld = [0.5079, 0.2239]

# The phase axis on which fleck will evaluate the light curves
# will be defined on [0, `max_phase`]:
max_phase = 10
phases = np.linspace(0, 2 * np.pi * max_phase, 30 * max_phase) * u.rad

n_spots = 5
spot_radius = 0.1         # Rspot/Rstar
min_latitude = 10         # deg
max_latitude = 30         # deg
stellar_inclination = 90  # deg
incs = stellar_inclination * u.deg * np.ones(n_spots)
lons, lats, radii, inc_stellar = generate_spots(min_latitude, max_latitude,
                                                spot_radius, n_spots=1,
                                                inclinations=incs)

star = Star(spot_contrast=spot_contrast, phases=phases, u_ld=u_ld)
lcs = star.light_curve(lons, lats, radii, inc_stellar)

phases_max = [10, 16, 20, 40, 60]

lcs2 = np.zeros_like(lcs)

for i, lc, phase_max in zip(range(n_spots), lcs.T, phases_max): 
    lc_evo = emerge_decay(phases, lc, phase_max, 5, 10)
    plt.plot(phases, lc_evo, label=f'spot {i}')
    
    lcs2[:, i] = lc_evo
    
plt.plot(phases, combine_lcs(lcs2), 'k', lw=2, label='all spots')
plt.legend()